In [71]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config

In [119]:
import torch.nn as nn
from transformers import GPT2Model

class GLAAttention(nn.Module):
    def __init__(self, Q, K, V):
        super().__init__()
        self.Q_weights = Q
        self.K_weights = K
        self.V_weights = V
        self.attention_weights = None  # Для сохранения attention weights

    def forward(self, x, layer_past=None, attention_mask=None,head_mask=None, use_cache=False, output_attentions=False):
       return (x,)
        


class GLA(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.config = self.gpt2.config
        self.gpt2_layers = []
        for i in range(12):
            tmp = self.gpt2.h[i]
            tmp.attn = GLAAttention(tmp.attn.c_attn.weight[:, :self.config.n_embd], 
                                     tmp.attn.c_attn.weight[:, self.config.n_embd:2*self.config.n_embd],
                                     tmp.attn.c_attn.weight[:, 2*self.config.n_embd:3*self.config.n_embd])
            self.gpt2_layers.append(tmp)

    def layers(self):
        return self.gpt2_layers, self.config

    def forward(self, X):
        for el in self.gpt2_layers:
            X = el.ln_1(X)
            X = el.attn(X)
            X = el.ln_2(X[0])
        return X  # Возвращаем результат после прохождения через все слои


model = GLA()
output = model(torch.ones((111, 768)))
print(output.shape)

torch.Size([111, 768])


In [111]:
layers = model.layers()
layers

([GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GLAAttention()
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  ),
  GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GLAAttention()
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  ),
  GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GLAAttention()
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  ),
  GPT2B

In [59]:
layers[1].attn.Q_weights

tensor([[-0.2906,  0.3057,  0.0302,  ...,  0.0132, -0.2652, -0.2058],
        [-0.3272,  0.2420,  0.2140,  ..., -0.0861,  0.0558,  0.1168],
        [-0.2679,  0.1188, -0.2670,  ..., -0.1061,  0.2167,  0.1066],
        ...,
        [-0.0284,  0.4304, -0.1394,  ..., -0.1750,  0.0154, -0.0614],
        [ 0.1730,  0.0967,  0.0262,  ...,  0.1744,  0.3897, -0.2129],
        [ 0.0422,  0.1598, -0.2512,  ..., -0.0259,  0.2618,  0.0779]],
       grad_fn=<SliceBackward0>)